In [1]:
import pandas as pd
import sys
sys.path.append('..')
sys.path.append('../..')
sys.path.append('../../..')

import os
from dotenv import load_dotenv
load_dotenv()

import ssl
ssl._create_default_https_context = ssl._create_stdlib_context

from helpers.focusmate import fm_sessions_data_to_df
from api.helpers.focusmate import fetch_all_focusmate_sessions, fetch_focusmate_profile
from api.helpers.request import get_access_token_from_db

fm_api_profile_endpoint = os.getenv("NEXT_PUBLIC_FM_API_PROFILE_ENDPOINT")
fm_api_sessions_endpoint = os.getenv("NEXT_PUBLIC_FM_API_SESSIONS_ENDPOINT")
session_id = ""
local_timezone = "America/New_York"

In [2]:
access_token = get_access_token_from_db(session_id)

profile_data = fetch_focusmate_profile(
        fm_api_profile_endpoint, access_token).get("user")
local_timezone: str = profile_data.get("timeZone")
member_since: str = profile_data.get("memberSince")

sessions_data = await fetch_all_focusmate_sessions(
    fm_api_sessions_endpoint, access_token, member_since)

all_sessions = fm_sessions_data_to_df(sessions_data, local_timezone)
sessions = all_sessions[all_sessions['completed'] == True].copy()

sessions_copy = sessions.copy()

In [3]:
sessions_copy['date'] = sessions_copy['start_time'].dt.date

sessions_copy.sort_values('date', inplace=True)

sessions_copy['day_over_day_delta'] = sessions_copy['date'].diff().dt.days

sessions_copy['consecutive_day_group_id'] = (sessions_copy['day_over_day_delta'] != 1).cumsum()

streak_lengths = sessions_copy.groupby('consecutive_day_group_id').size()

longest_streak = streak_lengths.max()

longest_streak_id = streak_lengths.idxmax()
longest_streak_dates = sessions_copy.loc[sessions_copy['consecutive_day_group_id'] == longest_streak_id, 'date']
longest_streak_start_date = longest_streak_dates.min()
longest_streak_end_date = longest_streak_dates.max()

print(longest_streak, (str(longest_streak_start_date), str(longest_streak_end_date)))
sessions_copy.tail(50)

4 ('2022-03-14', '2022-03-17')


,session_id,duration,start_time,requested_at,joined_at,completed,session_title,partner_id,date,day_over_day_delta,streak_id
1129,20ce7734-c875-4a80-9015-46eb6f4671b0,3000000,2024-05-14 08:15:00,2024-05-12 19:33:03.476481,2024-05-14 08:06:03.000076,True,,0016c312-8924-4510-857c-89d6669f6425,2024-05-14,0.0,1117
1130,6be22e8b-1265-495a-b5ec-10245bcd4cef,3000000,2024-05-14 07:15:00,2024-05-12 19:32:59.579414,2024-05-14 07:14:19.128616,True,,7266821a-ed03-46c2-9f96-6baab4303e57,2024-05-14,0.0,1118
1124,3b7cbe0e-7f08-4a4f-94eb-0158cf0c4048,3000000,2024-05-15 07:15:00,2024-05-15 07:11:31.522799,2024-05-15 07:14:43.622722,True,,f6e1434c-2c1a-42ba-9f18-6f3239b1497e,2024-05-15,1.0,1118
1123,65df6ba1-e3f9-4223-ae4a-3613db4d55c7,3000000,2024-05-15 17:00:00,2024-05-15 16:54:37.711303,2024-05-15 16:59:18.567495,True,,f689f4a7-ea23-4698-b23e-a0734568a640,2024-05-15,0.0,1119
1122,72fce5c6-916a-4de3-80de-172af9fa4f93,3000000,2024-05-16 07:15:00,2024-05-12 19:33:00.826546,2024-05-16 07:14:32.596720,True,,c5d92ced-7b14-401c-b975-7139e104f240,2024-05-16,1.0,1119
1121,07fc3d5e-8436-466b-9a47-c0bd224cf2a8,3000000,2024-05-16 08:15:00,2024-05-12 19:33:04.456432,2024-05-16 08:09:35.028556,True,,c5d92ced-7b14-401c-b975-7139e104f240,2024-05-16,0.0,1120
1120,c1a84dfd-6384-4ee4-a60f-d50b594599a7,3000000,2024-05-16 16:15:00,2024-05-16 16:08:16.062686,2024-05-16 16:15:24.340946,True,,0928b0a1-a56c-4a5d-901a-41e83465f5cd,2024-05-16,0.0,1121
1113,5e89989f-141b-46ee-b23c-db27adc6715f,3000000,2024-05-20 18:00:00,2024-05-20 14:47:36.114109,2024-05-20 17:59:41.872099,True,,28830810-7b8a-4162-ad25-66301074f8cc,2024-05-20,4.0,1122
1114,141f3e7e-d5d0-4509-8897-5b8aeb84daa4,3000000,2024-05-20 17:00:00,2024-05-20 14:47:34.515739,2024-05-20 16:59:11.461891,True,,28830810-7b8a-4162-ad25-66301074f8cc,2024-05-20,0.0,1123
1115,e93eaec0-92ec-4573-8b93-d2650e0ae80d,3000000,2024-05-20 16:00:00,2024-05-20 14:47:33.784627,2024-05-20 15:59:06.686351,True,,28830810-7b8a-4162-ad25-66301074f8cc,2024-05-20,0.0,1124
